In [3]:
import requests
from openpyxl import load_workbook

base_url = "https://www.gsar.ford.com/onlinereports/be/getVehInfoFull"

In [4]:
def get_token():
    return "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6ImFSZ2hZU01kbXI2RFZpMTdWVVJtLUJlUENuayJ9.eyJhdWQiOiJ1cm46Z3NhcjpyZXNvdXJjZTp3ZWI6cHJvZCIsImlzcyI6Imh0dHBzOi8vY29ycC5zdHMuZm9yZC5jb20vYWRmcy9zZXJ2aWNlcy90cnVzdCIsImlhdCI6MTcwNzM5ODE2MiwiZXhwIjoxNzA3NDI2OTYyLCJDb21tb25OYW1lIjoiVkRVQVJUMTAiLCJzdWIiOiJWRFVBUlQxMCIsInVpZCI6InZkdWFydDEwIiwiZm9yZEJ1c2luZXNzVW5pdENvZGUiOiJGU0FNUiIsImdpdmVuTmFtZSI6IlZpY3RvciIsInNuIjoiRHVhcnRlIiwiaW5pdGlhbHMiOiJWLiIsIm1haWwiOiJ2ZHVhcnQxMEBmb3JkLmNvbSIsImVtcGxveWVlVHlwZSI6Ik0iLCJzdCI6IkJBIiwiYyI6IkJSQSIsImZvcmRDb21wYW55TmFtZSI6IklOU1QgRVVWQUxETyBMT0RJIE4gUkVHSU9OQUwgQkFISUEiLCJmb3JkRGVwdENvZGUiOiIwNjY0Nzg0MDAwIiwiZm9yZERpc3BsYXlOYW1lIjoiRHVhcnRlLCBWaWN0b3IgKFYuKSIsImZvcmREaXZBYmJyIjoiUFJEIiwiZm9yZERpdmlzaW9uIjoiUEQgT3BlcmF0aW9ucyBhbmQgUXVhbGl0eSIsImZvcmRDb21wYW55Q29kZSI6IjAwMDE1ODM4IiwiZm9yZE1hbmFnZXJDZHNpZCI6Im1tYWdyaTEiLCJmb3JkTVJSb2xlIjoiTiIsImZvcmRTaXRlQ29kZSI6IjY1MzYiLCJmb3JkVXNlclR5cGUiOiJFbXBsb3llZSIsImFwcHR5cGUiOiJQdWJsaWMiLCJhcHBpZCI6InVybjpnc2FyOmNsaWVudGlkOndlYjpwcm9kIiwiYXV0aG1ldGhvZCI6Imh0dHA6Ly9zY2hlbWFzLm1pY3Jvc29mdC5jb20vd3MvMjAwOC8wNi9pZGVudGl0eS9hdXRoZW50aWNhdGlvbm1ldGhvZC93aW5kb3dzIiwiYXV0aF90aW1lIjoiMjAyNC0wMi0wOFQxMzoyMTowMi40NjhaIiwidmVyIjoiMS4wIn0.ZLJEWzeff_UG_cTHr5tZizQKmOaJBIJV0f5GGIQH5GYoag2uj9ALzodEzaooH0NdAKHUtS_LkOIGB7B-RgglJ6dITaqTKfnwi6NSpNrUhfsehDmus_byxlfR1HJdBMzz8zrYac_gO0Qr1R76fWGl8hhQCPzDz_GlARbUBSfNRHODBP1CSbbvFX_N3C803Ev4CyTKxjVWinFqCAM18S4_MTLKQ6uVKxM5PACfuhc49Cj9Q45G19P-pUjeFgec1j8IiYW-pgv7lyJiGx45JkImZbRn2R3nkf0Jyy4cSdeDm6cPygN9FuaRRO55DM0MasD0L_hIhvuKNNpqDP-5_7HEHg"

In [5]:
def get_full_vins():
    odi_vin_dict = {}

    try:
        workbook = load_workbook(
            filename="./../data/external/NSCCV-000502-20240205.xlsx"
        )
        sheet = workbook["VOQS"]

        for row in sheet.iter_rows(min_row=2):
            odi_id = row[0].value
            vin = row[1].value
            if odi_id is not None and vin is not None:
                odi_vin_dict[odi_id] = vin

    except FileNotFoundError as exc:
        raise exc

    return odi_vin_dict

In [6]:
def get_info_by_vin(vin:str, token:str) -> None:
    response = requests.get(base_url,
                            params={'vin':vin},
                            headers={"Authorization": f"Bearer {token}"},
                            proxies={'http':"http://internet.ford.com:83",'https':"http://internet.ford.com:83"}
                            )
    data_needed = ['wersVl', 'origWarantDate', 'prodDate', 'plant', 'globVl', 'awsVl']

    data = dict(response.json())
    for key in data_needed:
        print(f"{key}: {data.get(key)}")

In [7]:
token = get_token()
vins = get_full_vins()

data = [get_info_by_vin(vin, token) for vin in vins if vin[-5] != '*' and vin != 'N/A' and len(vin) != 17]

c:\Users\VDUART10\AppData\Local\Ford-Conda\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


ProxyError: HTTPSConnectionPool(host='www.gsar.ford.com', port=443): Max retries exceeded with url: /onlinereports/be/getVehInfoFull?vin=11569514 (Caused by ProxyError('Unable to connect to proxy', NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002887ED74700>: Failed to resolve 'internet.ford.com' ([Errno 11001] getaddrinfo failed)")))